In [ ]:
# Output Parser와 LCEL의 시작
# LLM의 출력은 항상 텍스트이지만, 우리는 구조화된 데이터가 필요합니다

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 일관된 결과를 위한 낮은 temperature 설정
chat = ChatOpenAI(temperature=0.1)

In [ ]:
# 커스텀 Output Parser 구현
# BaseOutputParser를 상속받아 원하는 형태로 텍스트를 변환

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    """쉼표로 구분된 텍스트를 리스트로 변환하는 파서"""
    
    def parse(self, text):
        """
        LLM의 텍스트 출력을 파싱하는 핵심 메서드
        예: "apple, banana, orange" → ["apple", "banana", "orange"]
        """
        # strip(): 양쪽 공백 제거, split(","): 쉼표로 분할
        items = text.strip().split(",")
        # 각 항목의 앞뒤 공백도 제거하여 깔끔한 리스트 반환
        return list(map(str.strip, items))

In [ ]:
# Output Parser용 프롬프트 템플릿
# LLM이 우리가 원하는 형태로 출력하도록 정확한 지시사항 제공

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 매우 구체적인 지시사항:
            # 1. 리스트 생성 머신 역할
            # 2. 쉼표로 구분된 리스트만 출력
            # 3. 최대 개수 제한
            # 4. 소문자로만 출력
            # 5. 다른 내용은 절대 포함하지 말 것
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        # 사용자의 실제 질문
        ("human", "{question}"),
    ]
)

In [ ]:
# 🎉 LCEL의 마법! LangChain Expression Language
# 파이프 연산자(|)로 체인을 우아하게 구성하는 혁명적인 방법

# 전통적 방법이었다면:
# 1. template.format_messages() 호출
# 2. chat.predict_messages() 호출  
# 3. parser.parse() 호출
# → 총 3번의 수동 호출 + 중간 변수 관리

# LCEL 방법: 한 줄로 끝!
chain = template | chat | CommaOutputParser()
# template → chat → CommaOutputParser 순서로 데이터가 흐름
# | 연산자는 LangChain이 특별히 구현한 마법의 연산자

# invoke(): 체인 전체를 실행하는 통합 메서드
# 딕셔너리를 입력하면 최종 파싱된 결과가 출력됨
result = chain.invoke({"max_items": 5, "question": "What are the pokemons?"})
result  # ['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']